## uvsub for Phase 2

For phase 2 files. As of 10/5/17, I only have 5 of these.

12/13/17: I have 18.

In [1]:
#imports
import numpy as np
import pandas as pd
import os
import sys
import subprocess
import matplotlib as plt
%matplotlib qt

In [2]:
#set to load in xlsx files(True) or txt files(False). Should return the same result either way.
xlsx=True

In [3]:
#load in files
files_raw = str(subprocess.check_output(['ls', 'Phase2/']))
files = files_raw.split('\\n')

#fix files
files[0] = files[0][2:]#remove "b' from beginning
files=files[:-1]#remove non-entry last item

filelist=[]
#get rid of xlsx files from file selection
if xlsx==False:
    for filename in files:
        if filename[-4:] == '.txt':
            filelist.append(filename)
#get rid of txt files from file selection
if xlsx==True:
    for filename in files:
        if filename[-4:] == 'xlsx':
            filelist.append(filename)
            
#last three observations have fucked frequencies, so getting rid of them here:
#filelist=filelist[:-3]
        
#manipulate names in files, create observations list
observations = []
for i in range(0,len(filelist)):
    if xlsx==False:
        observations.append(filelist[i][8:-15])
    if xlsx==True:
        observations.append(filelist[i][:-5])

obs_dfs = {}#dictionary of dataframes

#read in files
for obs in observations:
    if obs not in obs_dfs.keys():
        if xlsx==False:
            obs_dfs[obs] = pd.read_table('Phase2/13B-266.'+obs+'.flag_stats.txt',sep=' ')
            #fix each dataframe
            obs_dfs[obs].drop(['#total','%flagged'],axis=1,inplace=True)
            obs_dfs[obs].columns = ['freq','#flags','#total','%flagged']
            obs_dfs[obs].set_index('freq',inplace=True)
        if xlsx==True:
            obs_dfs[obs] = pd.read_excel('Phase2/'+obs+'.xlsx')
            #obs_dfs[obs].set_index('freq',inplace=True)
            
#clear some variables
files_raw=None
files=None
filelist=None

In [4]:
#need to set index, apply to each obs df
for obs in observations:
    obs_dfs[obs].index=np.arange(int(obs_dfs[obs].index.min()),int(obs_dfs[obs].index.min())+480,0.015625)

In [5]:
#check the new index cols
huh=[]
for obs in observations:
    #huh.append(obs_dfs[obs].index.min())#freq mininum. Should all be ~~~.6 
    huh.append(obs_dfs[obs].index.max()-obs_dfs[obs].index.min())#bandwidth. Should all be same
huh

[479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375,
 479.984375]

In [6]:
#save these to excel files if desired
for obs in observations:
    writer=pd.ExcelWriter(obs+'.xlsx',engine='xlsxwriter')
    obs_dfs[obs].to_excel(writer, sheet_name='Sheet1')
    writer.save()
    #FYI these will save to the same directory as the notebook

## Doing stuff with phase 2 data

In [6]:
#plotting #flags versus frequency
nplt=0
for obs in observations:
    if nplt==1:
        obs_dfs[obs].plot(y='%flagged',ax=ax,legend=False,xlim=(1111,1113))
    if nplt==0:
        ax = obs_dfs[obs].plot(y='%flagged',legend=False,xlim=(1111,1113))
        nplt=nplt+1

In [7]:
#add up all dataframes
sum_array=[]

In [8]:
#create tot_df: 30720*len(observations) dataframe of %flagged columns from each observation
count=0
for obs in observations:
    if count == 0:
        #init dataframe
        tot_df=obs_dfs[obs].drop(['#total','#flags'],axis=1)
        tot_df.columns = [str(count)]
        #tot_df=tot_df.set_index('index').index.name=None
        count=count+1
    if count >= 1:
        #append dataframes
        tot_df = pd.concat([tot_df,obs_dfs[obs].drop(['#total','#flags'],axis=1)],axis=1)
        count=count+1

#phase2total_df_norm: mean of %flagged from every observation for all frequencies (30720*1)
phase2total_df_norm = pd.DataFrame(tot_df.mean(axis=1))

In [26]:
#phase2total_df_norm: mean of %flagged from every observation for all frequencies (30720*1)
#phase2total_df_norm = pd.DataFrame(tot_df.mean(axis=1))

In [9]:
#see amount of freq channels for each observation
lengths=[]
for obs in observations:
    lengths.append(len(obs_dfs[obs]))
lengths

[30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720,
 30720]

In [10]:
#example dataframe for one observation
obs_dfs[obs]

,#flags,#total,%flagged
954.000000,1344200,1344200,1.000000
954.015625,1344200,1344200,1.000000
954.031250,1344200,1344200,1.000000
954.046875,1344200,1344200,1.000000
954.062500,1344200,1344200,1.000000
954.078125,1344200,1344200,1.000000
954.093750,1344200,1344200,1.000000
954.109375,1344200,1344200,1.000000
954.125000,1344200,1344200,1.000000
954.140625,1344200,1344200,1.000000


In [11]:
#save phase2total_df_norm:
writer=pd.ExcelWriter('phase2total_df_norm.xlsx',engine='xlsxwriter')
obs_dfs[obs].to_excel(writer, sheet_name='Sheet1')
writer.save()

## Load in phase 1 data to compare

In [12]:
#load phase1total_df.xlsx
phase1total_df = pd.read_excel('phase1total_df.xlsx')

In [30]:
# no need to run this cell
phase1_max = phase1total_df.max() #phase 1 data max, no need to run this cell

In [13]:
#function to apply to phase1total_df
norm_flip = lambda x: 1 - (x/43318941) #divide by theoretical max, 1 minus to fit phase 2 data

phase1total_df_norm = phase1total_df.apply(norm_flip,axis=1)
#phase1total_df_norm is normalized version of phase1total_df

#rename columns for plot legend
phase1total_df_norm.columns=['Phase 1']
phase2total_df_norm.columns=['Phase 2']

In [61]:
#save phase1total_df_norm:
writer=pd.ExcelWriter('phase1total_df_norm.xlsx',engine='xlsxwriter')
phase1total_df_norm.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [28]:
#create dictionary of normalized dataframes of phase 1 and 2, plot that shit
norm_dfs = {}
norm_dfs['phase1']=phase1total_df_norm
norm_dfs['phase2']=phase2total_df_norm
nplt=0
for df in norm_dfs.keys():
    if nplt==1:
        norm_dfs[df].plot(ax=ax,grid=True,yticks=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],legend=True)
    if nplt==0:
        ax = norm_dfs[df].plot(grid=True,yticks=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],legend=True)
        nplt=nplt+1
#phase1norm_phase2norm.tiff or For_CHILES/phase12_norm.tiff

In [62]:
len(phase1total_df_norm)

30464

In [19]:
norm_dfs['phase2'].plot(xlim=(1130,1131))

In [24]:
len(observations)

16

In [45]:
#Extra: plot For_CHILES/phase12_norm.tiff but only in the frequency ranges 1200-1222MHZ, 1260-1282MHz
ticks1200=list(range(1200,1223))
ticks1260=list(range(1260,1283))
nplt=0

for df in norm_dfs.keys():
    if nplt==1:
        norm_dfs[df][1200:1223].plot(ax=ax1200,grid=True,yticks=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],xticks=ticks1200,legend=True)
    if nplt==0:
        ax1200 = norm_dfs[df][1200:1223].plot(grid=True,yticks=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],xticks=ticks1200,legend=True)
        nplt=nplt+1
#phase12_norm_1200.tiff

In [46]:
#second plotting for above cell
nplt=0
for df in norm_dfs.keys():
    if nplt==1:
        norm_dfs[df][1260:1283].plot(ax=ax1260,grid=True,yticks=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],xticks=ticks1260,legend=True)
    if nplt==0:
        ax1260 = norm_dfs[df][1260:1283].plot(grid=True,yticks=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],xticks=ticks1260,legend=True)
        nplt=nplt+1
#phase12_norm_1260.tiff

In [48]:
observations

['sb30134968.eb30452820.57078.09859436343',
 'sb30134968.eb30465086.57081.090475219906',
 'sb30134968.eb30474259.57093.09921111111',
 'sb30134968.eb30477450.57096.05290840278',
 'sb30134968.eb30479375.57097.146173298606',
 'sb30134968.eb30483429.57099.04607552083',
 'sb30135053.eb30467755.57084.082227430554',
 'sb30135053.eb30508373.57105.020166180555',
 'sb30135053.eb30512680.57108.051298645834',
 'sb30135138.eb30514540.57110.0097362963',
 'sb30135138.eb30516312.57114.0210734375',
 'sb30135138.eb30519238.57115.0392883912',
 'sb30135308.eb30612630.57131.92938252314',
 'sb30135393.eb30614637.57132.979567685186',
 'sb30136049.eb30635090.57146.01679417824',
 'sb32005783.eb32150558.57527.97051119213',
 'sb32005783.eb32214781.57538.84848486111',
 'sb32005783.eb32219840.57539.92867947917']

## Create flagging percentage vs. flags amount for Phase 1 and 2

Create the same plot as npts_count, but with phase 2 and phase 1 data on same plot

Have to normalize each separately

#for each npts value, compute how many data points are above and below

In [ ]:
#phase 1 code
#copied from uvsub.ipynb and modified names 

#list of data created
npts_data=[]

#turn this array into ints
totdata=totdata.astype('int')

#now run through all possible npts values (resolution 100) and see how many freq data points are above
#add the npts_val and gcount to npts_data list
for npts_val in range(0,totdata.max(),100):
    npts_data.append([npts_val,sum(totdata > npts_val)])
#this takes a while, can reduce time and npts-resolution by increasing step size in for loop above
#for reference, totdata.max() = 33139217 

#turns resulting npts_data into dataframe and
#resets npts_data to save RAM
npts_df = pd.DataFrame(npts_data)
npts_data = []

#fixes index column
npts_df.set_index(0,inplace=True)